# 深度学习2：梯度

### 定义
对于n维向量 $X$ 和 $R^n$ 上的函数 $f(X)$，梯度定义为 $\nabla f(X) = \frac{\partial f}{\partial X} $

在理论中，我们要时刻注意梯度到底是什么：矩阵的梯度实质上就是函数族的梯度，可以通过计算每一个子函数的梯度间接得到。或者用我们的话来说，标量变量or向量变量。

### 理论
- 对 $A_{m × n}$ 都有 $\nabla (Ax) = A$ （$Ax$ 是超平面方程）
  
   $ \nabla Ax $ 实质上就是雅可比矩阵。$f_i = \sum_j a_{ij}x_j$ 故 $ J_{ij} = \frac{\partial f_i}{\partial x_j} = a_{ij}$ ，可得雅可比矩阵就是 $A$

- 对 $A_{n × m}$ 都有 $\nabla (x^TA) = A^T$

   这里按列分块：$g_i=\sum_j a_{ji}x_j$ 故 $J_{ij}^T = \frac{\partial g_i}{\partial x_j} = a_{ij}$ ，可得 $J = A^T$
  
- 对 $A_{n × n}$ 都有 $\nabla (x^TAx) = (A+A^T)x$
   
   记该二次型为 $f$，则 $f_{ij} = a_{ij}x_ix_j$ 即 $f=\sum_i\sum_j a_{ij}x_ix_j$ ，而 $(\nabla f)_k =\sum_i\sum_j\frac{ \partial  a_{ij}x_ix_j}{\partial {x_k}} = 
   \sum_i a_{ik} x_i + \sum_j a_{kj} x_j = g_k + f_k$ 
   
   因此 $\nabla f = (g_1+f_1,...,g_n+f_n)^T = (A+A^T)x$

- $\nabla \lVert \mathbf{x} \rVert^2=\nabla\mathbf{x}^T\mathbf{x}=2\mathbf{x}$

   由上式，$LHS=\nabla \mathbf{x^T} E \mathbf{x}  = Middle = 2Ex = 2x$

### 自动求导
原理：
- 计算图的概念
- 正向传播
- 反向传播
  
  空间复杂度小


以 $y = 2\mathbf{x}^T \mathbf{x}$ 为例：

0. 数学计算
   
   $y = 2 (x_1^2 + x_2^2 + x_3^2 + x_4^2) $

   $\nabla y = 4(x_1, ... ,x_4)$
1. 内存指定

In [66]:
import torch
x = torch.tensor([1.,1,4,5],requires_grad=True)
x.grad == None

True

2. 原函数

In [67]:
y = 2*torch.dot(x,x)

3. 求导

In [68]:
y.backward() # 反向计算求导
x.grad # y在x（给定）处的梯度

tensor([ 4.,  4., 16., 20.])

4. 清理内存

   从这里可以看出 `grad` 的值是累加的，因此需要手动清理内存。

In [69]:
z = x.sum()
z.backward()
x.grad

tensor([ 5.,  5., 17., 21.])

In [70]:
x.grad.zero_()

print(x.grad)
z.backward()
print(x.grad)

tensor([0., 0., 0., 0.])
tensor([1., 1., 1., 1.])


5. 分离计算
   
   人为地控制计算的流向

   - `detach()` 用于创建一个新的张量，这个张量与原张量共享数据内存，但与计算图分离。

- Graph示例
    ```
    z -->x
    \-->u-->x^2
    ```
- 数学推导

  若 `z-->u` 被分离：

  $z.sum = x_1 + x_2 + 16x_3 + 25x_4$

  $\nabla z.sum = (1,1,16,25)$

In [71]:
x.grad.zero_()
y = x * x
print(y)
u = y.detach()
z = u * x
x

tensor([ 1.,  1., 16., 25.], grad_fn=<MulBackward0>)


tensor([1., 1., 4., 5.], requires_grad=True)

In [72]:
z.sum().backward()
x.grad

tensor([ 1.,  1., 16., 25.])